In [ ]:
import pandas as pd
import numpy as np
import datetime
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# Ignorer certains warnings pour la lisibilité 
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn') # Pour les noms de features OneHotEncoder

print(f"Script démarré à: {datetime.datetime.now()}")

# --- 1. Chargement des Données Initiales ---
print("\n--- Chargement des Données ---")
lien_incident = 'LFB Incident data from 2018 onwards.csv'
lien_mobilisation = 'LFB Mobilisation data from 2021 - 2024.csv'
try:
    incidents_full = pd.read_csv(lien_incident, low_memory=False) # low_memory=False peut aider
    mobilisation_full = pd.read_csv(lien_mobilisation, low_memory=False)
    print("Fichiers CSV chargés.")
except FileNotFoundError:
    print("ERREUR: Fichiers CSV non trouvés.")
    exit()
except Exception as e:
    print(f"Erreur lors du chargement des CSV: {e}")
    exit()

# --- 2. Filtrage des Données (2024 Uniquement) ---
target_year = 2024
print(f"\n--- Filtrage pour l'année {target_year} ---")
incidents_2024 = incidents_full[incidents_full['CalYear'] == target_year].copy()
mobilisations_2024 = mobilisation_full[mobilisation_full['CalYear'] == target_year].copy()
print(f"Incidents {target_year}: {incidents_2024.shape[0]} lignes.")
print(f"Mobilisations {target_year}: {mobilisations_2024.shape[0]} lignes.")
del incidents_full, mobilisation_full # Libérer mémoire

# --- 3. Prétraitement Mobilisations ---
print("\n--- Prétraitement Mobilisations ---")
mobilisations_2024 = mobilisations_2024[mobilisations_2024['PumpOrder'] == 1].copy()
mobilisations_2024.drop_duplicates(inplace=True)
mobilisation_subset = mobilisations_2024[[
    'IncidentNumber', 'AttendanceTimeSeconds', 'DeployedFromLocation',
    'PumpOrder', 'DelayCodeId'
]].copy()
mobilisation_subset['DelayCodeId'] = mobilisation_subset['DelayCodeId'].fillna(1).astype(int)
print(f"Mobilisations traitées: {mobilisation_subset.shape[0]} lignes.")
del mobilisations_2024 # Libérer mémoire

# --- 4. Sélection et Fusion Incidents ---
print("\n--- Sélection/Fusion Incidents ---")
incident_cols_poc = [
    'IncidentNumber', 'DateOfCall', 'TimeOfCall', 'IncidentGroup',
    'StopCodeDescription', 'PropertyCategory', 'IncGeo_BoroughCode',
    'IncGeo_BoroughName', 'IncGeo_WardCode', 'IncGeo_WardName',
    'IncidentStationGround'
]
incidents_poc_subset = incidents_2024[incident_cols_poc].copy()
df_merged = pd.merge(
    left=mobilisation_subset, right=incidents_poc_subset,
    on='IncidentNumber', how='inner'
)
print(f"Dimensions après fusion: {df_merged.shape}")
del incidents_poc_subset, incidents_2024, mobilisation_subset # Libérer mémoire

# --- 5. Nettoyage Post-Fusion ---
print("\n--- Nettoyage Post-Fusion ---")
cols_to_check_na = [
    'AttendanceTimeSeconds', 'IncidentGroup', 'PropertyCategory',
    'IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_BoroughCode',
    'IncGeo_BoroughName', 'IncidentStationGround', 'DeployedFromLocation',
    'StopCodeDescription', 'DateOfCall', 'TimeOfCall'
]
rows_before = df_merged.shape[0]
df_poc_2024 = df_merged.dropna(subset=cols_to_check_na).copy()
print(f"{rows_before - df_poc_2024.shape[0]} lignes supprimées par dropna.")
df_poc_2024.drop_duplicates(inplace=True)
print(f"Dimensions après nettoyage: {df_poc_2024.shape}")
del df_merged # Libérer mémoire

# --- 6. Feature Engineering (Basique + Nouvelles Features) ---
print("\n--- Feature Engineering ---")
# Dates / Heures
df_poc_2024['DateOfCall'] = pd.to_datetime(df_poc_2024['DateOfCall'], errors='coerce')
df_poc_2024['TimeOfCall_dt'] = pd.to_datetime(df_poc_2024['TimeOfCall'], format='%H:%M:%S', errors='coerce')
rows_before = df_poc_2024.shape[0]
df_poc_2024.dropna(subset=['DateOfCall', 'TimeOfCall_dt'], inplace=True)
if rows_before > df_poc_2024.shape[0]: print(f"{rows_before - df_poc_2024.shape[0]} lignes suppr. (erreur date/heure).")

df_poc_2024['Year'] = df_poc_2024['DateOfCall'].dt.year # Gardé même si une seule année, pour cohérence
df_poc_2024['Month'] = df_poc_2024['DateOfCall'].dt.month
df_poc_2024['DayOfWeek'] = df_poc_2024['DateOfCall'].dt.dayofweek # Lundi=0
df_poc_2024['Hour'] = df_poc_2024['TimeOfCall_dt'].dt.hour

# Nouvelles Features Temporelles
df_poc_2024['IsWeekend'] = df_poc_2024['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
time_bins = [0, 6, 12, 18, 24] # Night, Morning, Afternoon, Evening
time_labels = ['Night', 'Morning', 'Afternoon', 'Evening']
df_poc_2024['TimeOfDay'] = pd.cut(df_poc_2024['Hour'], bins=time_bins, labels=time_labels, right=False, include_lowest=True)
print("Nouvelles features créées: Month, IsWeekend, TimeOfDay")

# Colonnes Fixes
df_poc_2024['PumpOrder'] = 1
df_poc_2024['DelayCodeId'] = df_poc_2024['DelayCodeId'].astype(int)

# --- 7. Création Mappings Nom -> Code ---
print("\n--- Création Mappings Nom -> Code ---")
categories_poc = {}
try:
    ward_mapping_df = df_poc_2024[['IncGeo_WardName', 'IncGeo_WardCode']].drop_duplicates().dropna()
    ward_mapping_df = ward_mapping_df.drop_duplicates(subset=['IncGeo_WardName'], keep='first')
    ward_name_to_code = pd.Series(ward_mapping_df['IncGeo_WardCode'].values, index=ward_mapping_df['IncGeo_WardName']).to_dict()
    categories_poc['ward_name_to_code'] = ward_name_to_code
    print(f"- Mapping Ward Nom -> Code créé ({len(ward_name_to_code)} entrées).")

    borough_mapping_df = df_poc_2024[['IncGeo_BoroughName', 'IncGeo_BoroughCode']].drop_duplicates().dropna()
    borough_mapping_df = borough_mapping_df.drop_duplicates(subset=['IncGeo_BoroughName'], keep='first')
    borough_name_to_code = pd.Series(borough_mapping_df['IncGeo_BoroughCode'].values, index=borough_mapping_df['IncGeo_BoroughName']).to_dict()
    categories_poc['borough_name_to_code'] = borough_name_to_code
    print(f"- Mapping Borough Nom -> Code créé ({len(borough_name_to_code)} entrées).")
except Exception as e:
    print(f"Erreur lors de la création des mappings: {e}")

# --- 8. Préparation pour Modélisation (Sélection, Encodage) ---
print("\n--- Préparation pour Modélisation ---")

# Colonnes à utiliser pour l'entraînement (incluant nouvelles features)
# Exclure Noms (WardName, BoroughName), DateOfCall, TimeOfCall, TimeOfCall_dt
features_for_model = [
    'Year', 'Month', 'DayOfWeek', 'Hour', 'IsWeekend', 'TimeOfDay', # Temporelles
    'IncidentGroup', 'StopCodeDescription', 'PropertyCategory', # Incident
    'IncGeo_BoroughCode', 'IncGeo_WardCode', 'IncidentStationGround', # Géo (Codes)
    'DeployedFromLocation', # Mobilisation
    'PumpOrder', 'DelayCodeId', # Fixes/Mobilisation
    'AttendanceTimeSeconds' # Cible
]
df_model_input = df_poc_2024[features_for_model].copy()
del df_poc_2024 # Libérer mémoire

# Encodage
label_encode_cols = ['IncGeo_BoroughCode', 'IncGeo_WardCode', 'IncidentStationGround']
# Inclure TimeOfDay dans OneHot, car catégoriel ordinal/cyclique mal géré par arbres parfois
one_hot_encode_cols = ['TimeOfDay', 'IncidentGroup', 'StopCodeDescription', 'PropertyCategory', 'DeployedFromLocation']

label_encoders_poc = {}
print("Encodage Label...")
for col in label_encode_cols:
    categories_poc[col] = df_model_input[col].astype(str).unique().tolist() # Sauvegarde CODES uniques
    le = LabelEncoder()
    df_model_input[col] = le.fit_transform(df_model_input[col].astype(str))
    label_encoders_poc[col] = le

print("Encodage One-Hot...")
# Configurer OneHotEncoder pour gérer les catégories inconnues potentiellement futures
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Sauvegarder catégories avant transformation
for col in one_hot_encode_cols:
     categories_poc[col] = df_model_input[col].astype(str).unique().tolist()

# Fitter et transformer
ohe.fit(df_model_input[one_hot_encode_cols].astype(str))
# Obtenir les noms de features générés par OHE
ohe_feature_names = ohe.get_feature_names_out(one_hot_encode_cols)
# Appliquer la transformation et créer un nouveau DataFrame
encoded_data = ohe.transform(df_model_input[one_hot_encode_cols].astype(str))
df_encoded_part = pd.DataFrame(encoded_data, columns=ohe_feature_names, index=df_model_input.index)

# Combiner avec les colonnes non-encodées (numériques et label-encoded)
df_encoded = pd.concat([df_model_input.drop(columns=one_hot_encode_cols), df_encoded_part], axis=1)
print(f"Dimensions après encodage: {df_encoded.shape}")

# --- 9. Définition X, y et Sauvegarde Colonnes ---
y_poc = df_encoded['AttendanceTimeSeconds']
X_poc = df_encoded.drop('AttendanceTimeSeconds', axis=1)
categories_poc['model_columns'] = list(X_poc.columns) # Sauvegarder l'ordre final exact

# --- 10. Train / Test Split ---
print("\n--- Séparation Train/Test ---")
X_train_poc, X_test_poc, y_train_poc, y_test_poc = train_test_split(
    X_poc, y_poc, test_size=0.2, random_state=42, stratify=df_model_input['Month']
)
print(f"Taille Train set PoC: {X_train_poc.shape}")
print(f"Taille Test set PoC: {X_test_poc.shape}")

# --- 11. Mise à l'échelle ---
print("\n--- Mise à l'échelle (StandardScaler) ---")
# Ne scaler que les colonnes numériques (pas les one-hot déjà en 0/1)
scaler_poc = StandardScaler()
X_train_poc_scaled = scaler_poc.fit_transform(X_train_poc)
X_test_poc_scaled = scaler_poc.transform(X_test_poc)
print("Mise à l'échelle terminée.")

# --- 12. Optimisation Hyperparamètres (RandomizedSearchCV) ---
print("\n--- Optimisation Hyperparamètres (RandomizedSearchCV) ---")
# Définir l'espace de recherche
param_distributions = {
    'n_estimators': [100, 200, 300, 400], # Nombre d'arbres
    'learning_rate': [0.01, 0.05, 0.1, 0.2], # Taux d'apprentissage
    'num_leaves': [20, 31, 40, 50], # Max leaves pour un arbre
    'max_depth': [-1, 10, 15, 20], # Profondeur max (-1 = illimité)
    'subsample': [0.7, 0.8, 0.9, 1.0], # Fraction d'échantillons pour entraîner chaque arbre
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0], # Fraction de features pour chaque arbre
    'reg_alpha': [0, 0.01, 0.1, 0.5], # Régularisation L1
    'reg_lambda': [0, 0.01, 0.1, 0.5] # Régularisation L2
}

# Initialiser le modèle de base
lgbm = LGBMRegressor(random_state=42, n_jobs=-1)

# Initialiser RandomizedSearchCV
n_iter_search = 50 # Nombre de combinaisons à tester 
random_search = RandomizedSearchCV(
    lgbm,
    param_distributions=param_distributions,
    n_iter=n_iter_search,
    scoring='r2', # Métrique à optimiser
    cv=5, # Nombre de folds pour Cross-Validation
    n_jobs=-1, # Utiliser tous les CPUs disponibles
    random_state=42,
    verbose=1 # Afficher la progression
)

print(f"Lancement de RandomizedSearchCV ({n_iter_search} itérations, CV=5)...")
# Entraîner la recherche sur les données d'entraînement mises à l'échelle
search_start_time = datetime.datetime.now()
random_search.fit(X_train_poc_scaled, y_train_poc)
search_end_time = datetime.datetime.now()
print(f"Recherche terminée en {search_end_time - search_start_time}.")

# Meilleurs paramètres et meilleur score CV
print(f"Meilleurs paramètres trouvés: {random_search.best_params_}")
print(f"Meilleur score R² (Validation Croisée): {random_search.best_score_:.4f}")

# Récupérer le meilleur modèle trouvé
best_model_poc = random_search.best_estimator_

# --- 13. Évaluation Finale (avec meilleur modèle) ---
print("\n" + "="*10 + " DÉBUT ÉVALUATION FINALE " + "="*10) 

print("\n--- Évaluation Finale (Meilleur Modèle) ---")
y_pred_poc_train = best_model_poc.predict(X_train_poc_scaled)
y_pred_poc_test = best_model_poc.predict(X_test_poc_scaled)

r2_train_poc = r2_score(y_train_poc, y_pred_poc_train)
r2_test_poc = r2_score(y_test_poc, y_pred_poc_test)
rmse_test_poc = np.sqrt(mean_squared_error(y_test_poc, y_pred_poc_test))

print(f"R-squared (R²) Train: {r2_train_poc:.4f}")
print(f"R-squared (R²) Test:  {r2_test_poc:.4f}")
print(f"RMSE Test:           {rmse_test_poc:.2f} secondes")

print("\n" + "="*10 + " FIN ÉVALUATION FINALE " + "="*10) # 

# --- 14. Sauvegarde des Artefacts Finaux ---
print("\n--- Sauvegarde des Artefacts pour Streamlit ---")
try:
    joblib.dump(best_model_poc, 'model_poc.joblib') # Sauvegarde du MEILLEUR modèle
    print("- model_poc.joblib sauvegardé (meilleur modèle trouvé).")
    joblib.dump(scaler_poc, 'scaler_poc.joblib')
    print("- scaler_poc.joblib sauvegardé.")
    joblib.dump(label_encoders_poc, 'label_encoders_poc.joblib')
    print("- label_encoders_poc.joblib sauvegardé.")
    # S'assurer que categories_poc contient tout ce qui est nécessaire (mappings, listes, cols)
    joblib.dump(categories_poc, 'categories_poc.joblib')
    print("- categories_poc.joblib sauvegardé.")
    print("\nSauvegarde terminée avec succès !")
except Exception as e:
    print(f"\nERREUR lors de la sauvegarde des fichiers : {e}")

print(f"\nScript terminé à: {datetime.datetime.now()}")

Script démarré à: 2025-04-11 10:43:02.745005

--- Chargement des Données ---
Fichiers CSV chargés.

--- Filtrage pour l'année 2024 ---
Incidents 2024: 134043 lignes.
Mobilisations 2024: 199046 lignes.

--- Prétraitement Mobilisations ---
Mobilisations traitées: 127628 lignes.

--- Sélection/Fusion Incidents ---
Dimensions après fusion: (127079, 15)

--- Nettoyage Post-Fusion ---
407 lignes supprimées par dropna.
Dimensions après nettoyage: (126670, 15)

--- Feature Engineering ---


/var/folders/vg/6lvcxy196lvc3r3m9gdnc_2m0000gn/T/ipykernel_88730/820241200.py:89: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_poc_2024['DateOfCall'] = pd.to_datetime(df_poc_2024['DateOfCall'], errors='coerce')


Nouvelles features créées: Month, IsWeekend, TimeOfDay

--- Création Mappings Nom -> Code ---
- Mapping Ward Nom -> Code créé (691 entrées).
- Mapping Borough Nom -> Code créé (33 entrées).

--- Préparation pour Modélisation ---
Encodage Label...
Encodage One-Hot...
Dimensions après encodage: (126670, 37)

--- Séparation Train/Test ---
Taille Train set PoC: (101336, 36)
Taille Test set PoC: (25334, 36)

--- Mise à l'échelle (StandardScaler) ---
Mise à l'échelle terminée.

--- Optimisation Hyperparamètres (RandomizedSearchCV) ---
Lancement de RandomizedSearchCV (50 itérations, CV=5)...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021408 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 523
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0

========== DÉBUT ÉVALUATION FINALE ==========



--- Évaluation Finale (Meilleur Modèle) ---

R-squared (R²) Train: 0.6568

R-squared (R²) Test:  0.6038

RMSE Test:           84.91 secondes



========== FIN ÉVALUATION FINALE ==========